In [7]:
import re
import datetime as dt
from datetime import date
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import csv

In [8]:
def getNumDays(begin_date):
    end_date = date(2018, 9, 26)
    delta = end_date - begin_date
    days_between = delta.days
    
    return(days_between)

415


In [ ]:
def getDate(x, begin_date):
    currentDate = begin_date + dt.timedelta(x)
    print(currentDate)
    return currentDate

In [17]:
def enterDate(date):
    selectMonth = driver.find_element_by_name('cpMonth')
    for option in selectMonth.find_elements_by_tag_name('option'):
        if(option.get_attribute('value') == str(date.month)):
            option.click()
            break
    
    selectYear = driver.find_element_by_name('cpYear')
    for option in selectYear.find_elements_by_tag_name('option'):
        if(option.get_attribute('value') == str(date.year)):
            option.click()
            break
           
    thisDay = str(date.day)   
    days = driver.find_elements_by_xpath("//a[@class='cpCurrentMonthDate']")
    for day in days:
        if(day.text == thisDay):
            day.click()
            break

In [ ]:
def formatRow(cols):
    newCols = []
    
    newCols.append(str(cols[0]))
    newCols.append(re.search('\d+/\d+/\d+', str(cols[1])).group(0))
    newCols.append(re.search('\d+:\d+', str(cols[1])).group(0))
    newCols.append(str(cols[2]))
    if(cols[2] == 'Accident'):
        newCols.append(re.search('Accident #: \d+', str(cols[3])).group(0))
        newCols.append('')
        newCols.append('')
        newCols.append('')
        newCols.append('')
    elif cols[2] == 'Arrest':
        newCols.append('')
        try:
            newCols.append(re.search('Arrestee: .+, .*[^Charge:]', str(cols[3])).group(0))
        except:
            newCols.append('Arrestee: Expunged,')
        newCols.append(re.search('Charge: .+', str(cols[3])).group(0))
        newCols.append('')
        newCols.append('')
    elif cols[2] == 'Incident':
        newCols.append('')
        newCols.append('')
        newCols.append('')
        newCols.append(re.search('Case #: \d+', str(cols[3])).group(0))
        newCols.append(re.search('Primary Offense: .+', str(cols[3])).group(0))
    newCols.append(str(cols[4]))
    
    cleanCols = []
    for item in newCols:
        cleanCols.append(item.strip())
    
    return cleanCols

In [ ]:
def getNumPages():
    try:
        total_pages = driver.find_element_by_id('mainContent_lblPageCount')
        total_pages = total_pages.text
        total_pages = re.search('of \d+', total_pages).group(0)
        total_pages = int(re.search('\d+', total_pages).group(0))
        return total_pages
    except:
        return 1

In [ ]:
def handleTable(numPages, file, writer):
    for i in range(1,numPages+1):
        stripedRows = driver.find_elements_by_xpath("//tr[@class='table-striped-row']")
        for row in stripedRows:
            newRow = []
            values = row.find_elements_by_tag_name('td')
            for value in values:
                newRow.append(value.text)
            strRow = formatRow(newRow)
            writer.writerow(strRow)

        nonstripedRows = driver.find_elements_by_xpath("//tr[@class='table-nonstriped-row']")
        for row in nonstripedRows:
            newRow = []
            values = row.find_elements_by_tag_name('td')
            for value in values:
                newRow.append(value.text)
            strRow = formatRow(newRow)
            writer.writerow(strRow)
            
        if(i < numPages):    
            link = driver.find_element_by_link_text(str(i+1))
            link.click()

In [ ]:
days = getNumDays()
starting_page = 'http://p2c.chpd.us/Summary_Disclaimer.aspx'
street_num = "207"
street = "SOUTH ROAD"
begin_date = date(2017, 8, 1)
end_date = '9/20/2018'

for i in days:
    

In [ ]:
path = 'CH_reports.csv'
headers = ['Event', 'Date', 'Time', 'Type','Accident_Number', 'Arrestee', 'Charge', 
           'Case_Number','Primary_Offense', 'Location']
with open(path, 'w') as csvFile:
    writer = csv.writer(csvFile, delimiter=',')
    writer.writerow(headers)

    starting_page = 'https://p2c.police.unc.edu/Summary_Disclaimer.aspx'
    begin_date = date(2017, 8, 1)
    days = getNumDays(begin_date)

    driver = webdriver.Chrome(executable_path='/Users/radair/Documents/chromedriver')
    driver.implicitly_wait(60)
    driver.get(starting_page)

    driver.find_element_by_id('mainContent_btnContinue').click()

    for i in range(0,days):
        current_date = getDate(i, begin_date)
        fromDate = driver.find_element_by_id('mainContent_txtDateFrom_imgDatePicker')
        fromDate.click()
        enterDate(current_date)

        toDate = driver.find_element_by_id('mainContent_txtDateTo_imgDatePicker')
        toDate.click()
        enterDate(current_date)

        driver.find_element_by_id('mainContent_cmdSubmit').click()

        numPages = getNumPages()
        handleTable(numPages, csvFile, writer)

        driver.find_element_by_id('mainContent_btnReset').click()